#Predicting on Google Books
<strong>Author:</strong> Nicholas Hunt-Walker<br/>
<strong>Desires:</strong>
- Get a list of words from the [top english verbs](http://www.acme2k.co.uk/acme/3star%20verbs.htm)
- For each word, search google blooks
- Organize books into a sensible schema
- Test classifiers on books to predict

In [1]:
import urllib2
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
from admin.admin import booksAPIkey
import time

In [2]:
words_url = "http://www.acme2k.co.uk/acme/3star%20verbs.htm"
words_page = urllib2.urlopen(words_url)

In [4]:
soup = bs(words_page)
words = soup.findAll("font", attrs={"color":"#0000CC"})
word_list = [str(word.get_text()) for word in words]

In [5]:
my_books = []
title_list = []
subject_stats = {}

In [6]:
def filter_books(retrieved_booklist, master_booklist):
    for item in retrieved_booklist["items"]:
        if ("averageRating" in item["volumeInfo"].keys()) & (item["volumeInfo"]["title"] not in title_list):
#         if item["volumeInfo"]["title"] not in title_list:
            master_booklist.append(item)
            title_list.append(item["volumeInfo"]["title"])
    
    return master_booklist

In [7]:
def get_data(idx, amt_per_page, keyword):
    the_url = "https://www.googleapis.com/books/v1/volumes?q={0}&maxResults={1}&printType=books&startIndex={2}&key={3}".format(keyword, amt_per_page, idx, booksAPIkey)
    the_json_info = urllib2.urlopen(the_url)
    data = json.loads(the_json_info.read())
    time.sleep(5)
    return data

In [8]:
def download_books(keyword, master_booklist):
    per_page = 40
    data = get_data(0, 10, keyword)
    book_total = data["totalItems"]

    if keyword not in subject_stats.keys():
        print keyword
        subject_stats[keyword] = book_total

    max_pages = book_total / per_page + 1
    for ii in range(20):
        data = get_data((ii*per_page) + 1, per_page, keyword)
        master_booklist = filter_books(data, master_booklist)
    
    return master_booklist

In [9]:
for keyword in word_list[:10]:
    my_books = download_books(keyword, my_books)

accept
care
enjoy
happen


HTTPError: HTTP Error 403: Forbidden

In [10]:
len(my_books)

1172

Alright so I can ping google books and get stuff. Next steps:
- get books for every word in my verb list
- get results into pymongo
- also save list as a text file, just in case